# ANOVA F-value For Feature Selection

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2020-01-01'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2014-01-02,3.95,3.95,3.98,3.84,3.85,20548400
2014-01-03,4.00,4.00,4.00,3.88,3.98,22887200
2014-01-06,4.13,4.13,4.18,3.99,4.01,42398300
2014-01-07,4.18,4.18,4.25,4.11,4.19,42932100
2014-01-08,4.18,4.18,4.26,4.14,4.23,30678700


In [3]:
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()

In [4]:
dataset.head()

,Adj Close,Close,High,Low,Open,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,
2014-01-03,4.00,4.00,4.00,3.88,3.98,22887200,1,1,1,0.012658
2014-01-06,4.13,4.13,4.18,3.99,4.01,42398300,1,1,1,0.032500
2014-01-07,4.18,4.18,4.25,4.11,4.19,42932100,0,1,0,0.012106
2014-01-08,4.18,4.18,4.26,4.14,4.23,30678700,0,0,0,0.000000
2014-01-09,4.09,4.09,4.23,4.05,4.20,30667600,0,0,1,-0.021531


In [5]:
dataset.shape

(1509, 10)

In [6]:
X = dataset[['Open', 'High', 'Low', 'Volume', 'Increase_Decrease', 'Buy_Sell_on_Open', 'Buy_Sell', 'Returns']]
y = dataset['Adj Close']

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [8]:
# Create an SelectKBest object to select features with two best ANOVA F-Values
fvalue_selector = SelectKBest(f_classif, k=2)

# Apply the SelectKBest object to the features and target
X_kbest = fvalue_selector.fit_transform(X, y)

In [9]:
print('Original number of features:', X.shape[1])
print('Reduced number of features:', X_kbest.shape[1])

Original number of features: 8
Reduced number of features: 2


In [10]:
fvalue_selector.scores_

array([2.30283497e+03, 5.57665017e+03, 8.27145665e+03, 5.78529879e+00,
       9.94101142e-01, 9.51461780e-01, 8.94105495e-01, 1.19069927e+00])

In [11]:
fvalue_selector.pvalues_

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.39746085e-94,
       5.33887423e-01, 7.46851758e-01, 9.31934761e-01, 1.14544123e-02])

In [12]:
names = X.columns.values[fvalue_selector.get_support()]
scores = fvalue_selector.scores_[fvalue_selector.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
#Sort the dataframe for better visualization
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)

  Feat_names     F_Scores
1        Low  8271.456648
0       High  5576.650173
